# Anthropic Claude Memory Tool + Gaugid Integration

This notebook demonstrates how to use `GaugidMemoryTool` with Anthropic's memory tool, allowing Claude to store and retrieve information across conversations using Gaugid profiles.

**Installation**: `pip install gaugid[anthropic]`

Based on: https://platform.claude.com/docs/en/agents-and-tools/tool-use/memory-tool

In [ ]:
import os

try:
    from anthropic import Anthropic
    from gaugid.integrations.anthropic import GaugidMemoryTool
    ANTHROPIC_AVAILABLE = True
except ImportError:
    ANTHROPIC_AVAILABLE = False
    print("⚠️  Anthropic SDK not installed. Install with: pip install gaugid[anthropic]")

# Set your API keys
os.environ["ANTHROPIC_API_KEY"] = "your-key-here"
os.environ["GAUGID_CONNECTION_TOKEN"] = "your-token-here"

In [ ]:
# Create Gaugid memory tool
print("1️⃣ Creating GaugidMemoryTool...")
memory_tool = GaugidMemoryTool(
    connection_token=os.getenv("GAUGID_CONNECTION_TOKEN"),
    namespace_prefix="claude",
    memory_type="episodic",
)
print("   ✅ GaugidMemoryTool created")

In [ ]:
# Create Anthropic client
client = Anthropic(api_key=os.getenv("ANTHROPIC_API_KEY"))
print("2️⃣ Claude will automatically use Gaugid for memory storage")

In [ ]:
# Example: Store a memory
print("3️⃣ Storing a memory...")
response1 = client.beta.messages.create(
    model="claude-sonnet-4-5",
    max_tokens=1024,
    messages=[
        {
            "role": "user",
            "content": "Remember that I prefer dark mode for all applications."
        }
    ],
    tools=[memory_tool],
    betas=["context-management-2025-06-27"],
)

# Process tool calls
for content_block in response1.content:
    if content_block.type == "text":
        print(f"   Claude: {content_block.text}")
    elif content_block.type == "tool_use":
        if content_block.name == "memory":
            print(f"   🔧 Claude using memory tool: {content_block.input.get('command', 'unknown')}")

In [ ]:
# Example: Retrieve the memory
print("4️⃣ Retrieving stored memory...")
response2 = client.beta.messages.create(
    model="claude-sonnet-4-5",
    max_tokens=1024,
    messages=[
        {
            "role": "user",
            "content": "What are my preferences for application themes?"
        }
    ],
    tools=[memory_tool],
    betas=["context-management-2025-06-27"],
)

for content_block in response2.content:
    if content_block.type == "text":
        print(f"   Claude: {content_block.text}")
    elif content_block.type == "tool_use":
        if content_block.name == "memory":
            print(f"   🔧 Claude using memory tool: {content_block.input.get('command', 'unknown')}")

an we avoid pr